In [ ]:
import ratemyprofessor

In [177]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from pathlib import Path

url = "/Users/scott/Downloads/class.html"
url = Path(url)

driver = webdriver.Chrome()
driver.implicitly_wait(10)
# file_url = 'file://' + url.as_posix()
# driver.get(file_url)
driver.get("https://catalog.apps.asu.edu/catalog/classes/classlist?campusOrOnlineSelection=C&honors=F&promod=F&searchType=all&subject=MAT&term=2247")

elements = driver.find_elements(By.CSS_SELECTOR, '.class-results-cell.instructor')

professor_names = []
for element in elements:
    try:
        # 尝试在当前教授元素中查找<a>标签
        link_element = element.find_element(By.TAG_NAME, 'a')
        # 获取<a>标签的文本，即教授的名字
        professor_name = link_element.text
        professor_names.append(professor_name)

    except NoSuchElementException:
        # 如果没有找到<a>标签，输出提示信息
        # print("未找到<a>标签在以下元素中:", element.text)
        pass
driver.quit()

AttributeError: 'NoneType' object has no attribute 'text'

In [174]:
professor_names = list(set(professor_names))

In [175]:
professor_names

[]

In [ ]:
asu = ratemyprofessor.get_school_by_name("Arizona State University")

In [ ]:
professor_ranking_list = []

for name in professor_names:
    print("Getting: " + name)
    professor = ratemyprofessor.get_professor_by_school_and_name(asu, name)
    professor_ranking_list.append([name, professor.rating, professor.num_ratings])

In [162]:
df = pd.DataFrame(professor_ranking_list)
df.columns = ['name', 'rating', 'rating_num']

In [165]:
df.sort_values(by='rating', ascending=False)

,name,rating,rating_num
30,Natalie Welcome,5.0,37
18,Michael Little Crow,4.9,122
25,Jenifer Boshes,4.8,68
14,Kayla Lock,4.8,39
12,Toan Vo,4.8,30
8,Matthew Weaver,4.8,17
19,Elizabeth Jones,4.7,25
27,Steven Brownstein,4.5,124
22,Sultan Al-Suleiman,4.5,206
21,Douglas Williams,4.3,52
